In [ ]:
import tensorflow as tf

n_inputs = X.shape[1]
n_hidden1 = 300
n_hidden2 = 300
n_hidden3 = 300
n_hidden4 = 812
n_hidden5 = 812
n_outputs = 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
# with tf.name_scope("dnn"):
#     hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
#                            activation=tf.nn.relu)
#     hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
#                            activation=tf.nn.relu)
#     hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3",
#                            activation=tf.nn.relu)
#     hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4",
#                            activation=tf.nn.relu)
#     hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5",
#                            activation=tf.nn.relu)
#     logits = neuron_layer(hidden5, n_outputs, name="outputs")

In [ ]:
tf.reset_default_graph()

In [ ]:
from tensorflow.contrib.layers import fully_connected

def leaky_relu(z, name=None):
    return tf.maximum(0.01*z, z, name=name)

with tf.name_scope("dnn"):

    hidden1 = fully_connected(X, n_hidden1, activation_fn=leaky_relu, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, activation_fn=leaky_relu, scope="hidden2")
    hidden3 = fully_connected(hidden2, n_hidden3, activation_fn=leaky_relu, scope="hidden3")
    logits = fully_connected(hidden3, n_outputs, activation_fn=None, scope="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs=15
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train)//batch_size):
            start_idx = iteration*batch_size
            end_idx = start_idx+batch_size
            X_batch, y_batch = X_train[start_idx:end_idx], Y_train[start_idx:end_idx]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: Y_test})
        print(epoch, "train accuracy:", acc_train, "Test accuracy:", acc_test)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = logits.eval(feed_dict={X: X_test})
    Y_pred = np.argmax(Z, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix
print("ROC: " + str(roc_auc_score(Y_test.astype(int), Y_pred)))
confusion_matrix(Y_test.astype(int), Y_pred)